<a href="https://colab.research.google.com/github/Jibrangit/DL_Assignments/blob/HW6/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import numpy as np
import matplotlib.pyplot as plt

y_load = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/ages.npy')
x_load = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/faces.npy')
dataset_size = len(x_load)

import random
def random_rearrange (X_tr, y_tr, seed):
    np.random.seed(seed)
    np.random.shuffle(X_tr)
    np.random.seed(seed)
    np.random.shuffle(y_tr)

no_data = int(x_load.shape[0])
train_size = int(0.8*0.8*no_data)
test_size = int(0.2*no_data)

print(y_load[0])   

random_rearrange(x_load, y_load, 0)
print(y_load[0])                        #Just checking if shuffling ahs been done coorrectly.

(x_test, x_train, x_valid) = x_load[:test_size], x_load[test_size:test_size+train_size],  x_load[test_size+train_size:]
(y_test, y_train, y_valid) = y_load[:test_size], y_load[test_size:test_size+train_size], y_load[test_size+train_size:]

w, h = 48, 48
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

40.0
48.0


In [22]:
model = Sequential()

In [23]:
# first part has two conv64 with a pooling layer

model.add(Conv2D(input_shape=(48, 48, 1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="sigmoid")) 
##units are 1 because we are using binary activation

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 12, 12, 256)      

In [25]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpointer])

  7/150 [>.............................] - ETA: 8:41 - loss: -806686789129.5332 - accuracy: 0.0000e+00